<a href="https://colab.research.google.com/github/BYRic-F/E-commerce-Fraud-detection/blob/main/Vesta%20E-commerce_Fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("hello world test")

hello world test
